# Summary

demostrating how to use external P(k) from class.

In [ ]:
from classy import Class

In [ ]:
import os
import io

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
def read_txt(path):
    with open(path, 'r') as f:
        # remove beginning `#`
        text = f.read()[1:]
    with io.StringIO(text) as f:
        # input has 6 sig. fig. which fits in float32
        return pd.read_csv(f, delim_whitespace=True, index_col=0, dtype=np.float32)

In [ ]:
def read_class_txt(path, camb=False):
    # read once
    with open(path, 'r') as f:
        text = f.read()

    # get last comment line
    comment = None
    for line in text.split('\n'):
        if line.startswith('#'):
            comment = line
    # remove beginning '#'
    comment = comment[1:]
    # parse comment line: get name after ':'
    names = [name.split(':')[1] for name in comment.strip().split()]

    with io.StringIO(text) as f:
        df = pd.read_csv(f, delim_whitespace=True, index_col=0, comment='#', header=None, names=names)
        return df if camb else df * 1.e12

In [ ]:
def plot_compare(df1, df2, keys, log=False, relative=False):
    '''plot to compare columns from `df1` and `df2`,
    using columns from `df1`.

    `keys`: tuple of str of names of `df1` and `df2`.
    `relative`: if True, plot the relative error w.r.t. `df1` instead.
    '''
    for col in df1.columns:
#         pd.merge(df1[col], df2[col], left_index=True, right_index=True).plot()
        df_temp = pd.concat((df1[col], df2[col]), axis=1, join='inner', keys=(' '.join((key, col)) for key in keys))
        if relative:
            temp = df_temp.values
            if log:
                plt.loglog(df_temp.index, np.abs((temp[:, 0] - temp[:, 1]) / temp[:, 0]), label=col)
            else:
                plt.plot(df_temp.index, np.abs((temp[:, 0] - temp[:, 1]) / temp[:, 0]), label=col)
            plt.legend()
            plt.show()
        else:
            if log:
                df_temp.plot(logx=True, logy=True)
            else:
                df_temp.plot()

In [ ]:
!tail -n 15 ~/git/fork/class/base_2018_plikHM_TTTEEE_lowl_lowE_lensing_custom_external.ini

In [ ]:
df = read_class_txt(os.path.expanduser('~/git/fork/class/output/base_2018_plikHM_TTTEEE_lowl_lowE_lensing_camb_cl_lensed.dat'), camb=True)

In [ ]:
df2 = read_class_txt(os.path.expanduser('~/git/fork/class/output/base_2018_plikHM_TTTEEE_lowl_lowE_lensing_camb_external_cl_lensed.dat'), camb=True)

In [ ]:
plot_compare(df, df2, ('direct', 'external'), relative=False)